In [27]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torch.nn.functional as F

In [28]:
# Define the custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.dataset = ImageFolder(root_dir, transform=transform)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx]

In [29]:
# Define the CNN model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout2d(0.25)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 6 * 6, 128)
        self.dropout_fc = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 3)  # Adjusted for 3 classes
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.dropout_fc(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

In [31]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [32]:
# Data transformations
transform = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                transforms.Resize((24, 24)),
                                transforms.ToTensor()])

In [33]:
# Create custom datasets
train_dataset = CustomDataset(root_dir='data/train', transform=transform)
valid_dataset = CustomDataset(root_dir='data/valid', transform=transform)

# Create data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=32, shuffle=True)

In [34]:
# Initialize the model, loss function, and optimizer
model = CNNModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [35]:
print(labels)

tensor([3, 2, 1, 1, 2, 3, 0, 1, 3, 0, 3, 2, 1, 2, 3, 2, 2, 2, 2, 2, 2, 0, 0, 0,
        2, 0, 1, 1, 2, 1, 1, 2])


In [36]:
print (outputs)

tensor([[0.3644, 0.3104, 0.3252],
        [0.3593, 0.3167, 0.3241],
        [0.3648, 0.3123, 0.3229],
        [0.3563, 0.3013, 0.3424],
        [0.3619, 0.3192, 0.3189],
        [0.3450, 0.3151, 0.3399],
        [0.3622, 0.3093, 0.3286],
        [0.3696, 0.2961, 0.3344],
        [0.3423, 0.3314, 0.3263],
        [0.3668, 0.2970, 0.3363],
        [0.3477, 0.3357, 0.3165],
        [0.3730, 0.3144, 0.3126],
        [0.3460, 0.3072, 0.3468],
        [0.3445, 0.3095, 0.3460],
        [0.3583, 0.3205, 0.3212],
        [0.3530, 0.3057, 0.3413],
        [0.3591, 0.3088, 0.3321],
        [0.3658, 0.3052, 0.3291],
        [0.3689, 0.2984, 0.3328],
        [0.3608, 0.3106, 0.3286],
        [0.3711, 0.3039, 0.3250],
        [0.3789, 0.2907, 0.3304],
        [0.3654, 0.3052, 0.3294],
        [0.3443, 0.3230, 0.3326],
        [0.3586, 0.3075, 0.3339],
        [0.3597, 0.3149, 0.3255],
        [0.3673, 0.3095, 0.3232],
        [0.3593, 0.3001, 0.3406],
        [0.3539, 0.3003, 0.3459],
        [0.364

In [37]:
# Training loop
epochs = 15
for epoch in range(epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Adjust labels to be within the valid range
        labels = torch.clamp(labels, max=2)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Epoch {epoch+1}/{epochs}, Validation Accuracy: {accuracy:.4f}')

Epoch 1/15, Validation Accuracy: 0.5727
Epoch 2/15, Validation Accuracy: 0.6859
Epoch 3/15, Validation Accuracy: 0.7044
Epoch 4/15, Validation Accuracy: 0.7159
Epoch 5/15, Validation Accuracy: 0.7275
Epoch 6/15, Validation Accuracy: 0.7298
Epoch 7/15, Validation Accuracy: 0.7206
Epoch 8/15, Validation Accuracy: 0.7229
Epoch 9/15, Validation Accuracy: 0.7321
Epoch 10/15, Validation Accuracy: 0.7321
Epoch 11/15, Validation Accuracy: 0.7206
Epoch 12/15, Validation Accuracy: 0.7229
Epoch 13/15, Validation Accuracy: 0.7252
Epoch 14/15, Validation Accuracy: 0.7252
Epoch 15/15, Validation Accuracy: 0.7344


In [38]:
#save the model
torch.save(model.state_dict(), 'model.pth')